In [17]:
import numpy as np
import pandas as pd
from pandas import Timestamp
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report, confusion_matrix ,accuracy_score 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
import matplotlib.gridspec as gridspec 

In [18]:
data_Train =pd.read_csv("KDDTrain+.txt")
data_Test =pd.read_csv("KDDTest+.txt")

In [19]:
columns = (['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent'
            ,'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root'
            ,'num_file_creations','num_shells','num_access_files','num_outbound_cmds','is_host_login'
            ,'is_guest_login','count','srv_count','serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate'
            ,'same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count','dst_host_srv_count'
            ,'dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate'
            ,'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate'
            ,'dst_host_srv_rerror_rate','attack','outcome'])


In [20]:
data_Train.columns=columns
data_Test.columns=columns


In [21]:
data_Train.drop(columns='outcome',axis=1, inplace=True )

In [22]:
data_Test.drop(columns='outcome',axis=1, inplace=True )

In [23]:
data_Train['attack'].value_counts()

normal             67342
neptune            41214
satan               3633
ipsweep             3599
portsweep           2931
smurf               2646
nmap                1493
back                 956
teardrop             892
warezclient          890
pod                  201
guess_passwd          53
buffer_overflow       30
warezmaster           20
land                  18
imap                  11
rootkit               10
loadmodule             9
ftp_write              8
multihop               7
phf                    4
perl                   3
spy                    2
Name: attack, dtype: int64

In [24]:
data_Test['attack'].value_counts()


normal             9711
neptune            4656
guess_passwd       1231
mscan               996
warezmaster         944
apache2             737
satan               735
processtable        685
smurf               665
back                359
snmpguess           331
saint               319
mailbomb            293
snmpgetattack       178
portsweep           157
ipsweep             141
httptunnel          133
nmap                 73
pod                  41
buffer_overflow      20
multihop             18
named                17
ps                   15
sendmail             14
rootkit              13
xterm                13
teardrop             12
xlock                 9
land                  7
xsnoop                4
ftp_write             3
worm                  2
loadmodule            2
perl                  2
sqlattack             2
udpstorm              2
phf                   2
imap                  1
Name: attack, dtype: int64

In [25]:
data_Test

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,tcp,private,REJ,0,0,0,0,0,0,...,1,0.00,0.06,0.00,0.00,0.00,0.0,1.00,1.00,neptune
1,2,tcp,ftp_data,SF,12983,0,0,0,0,0,...,86,0.61,0.04,0.61,0.02,0.00,0.0,0.00,0.00,normal
2,0,icmp,eco_i,SF,20,0,0,0,0,0,...,57,1.00,0.00,1.00,0.28,0.00,0.0,0.00,0.00,saint
3,1,tcp,telnet,RSTO,0,15,0,0,0,0,...,86,0.31,0.17,0.03,0.02,0.00,0.0,0.83,0.71,mscan
4,0,tcp,http,SF,267,14515,0,0,0,0,...,255,1.00,0.00,0.01,0.03,0.01,0.0,0.00,0.00,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22538,0,tcp,smtp,SF,794,333,0,0,0,0,...,141,0.72,0.06,0.01,0.01,0.01,0.0,0.00,0.00,normal
22539,0,tcp,http,SF,317,938,0,0,0,0,...,255,1.00,0.00,0.01,0.01,0.01,0.0,0.00,0.00,normal
22540,0,tcp,http,SF,54540,8314,0,0,0,2,...,255,1.00,0.00,0.00,0.00,0.00,0.0,0.07,0.07,back
22541,0,udp,domain_u,SF,42,42,0,0,0,0,...,252,0.99,0.01,0.00,0.00,0.00,0.0,0.00,0.00,normal


In [26]:
# Assume data_train_with_features_selected and data_test_with_features_selected are your dataframes


# Save data_test_with_features_selected to a CSV file
data_Test.to_csv('data_test.csv', index=False)

print("CSV files have been saved successfully.")

CSV files have been saved successfully.


In [10]:
def change_label(df):
    data_Train.attack.replace(['apache2', 'back', 'land', 'neptune', 'mailbomb', 'pod', 'processtable', 'smurf', 'teardrop',
                       'udpstorm', 'worm'], 'Dos', inplace=True)
    data_Train.attack.replace(['ftp_write', 'guess_passwd', 'httptunnel', 'imap', 'multihop', 'named', 'phf', 'sendmail',
                       'snmpgetattack', 'snmpguess', 'spy', 'warezclient', 'warezmaster', 'xlock', 'xsnoop'], 'R2L', inplace=True)
    data_Train.attack.replace(['ipsweep', 'mscan', 'nmap', 'portsweep', 'saint', 'satan'], 'Probe', inplace=True)
    data_Train.attack.replace(['buffer_overflow', 'loadmodule', 'perl', 'ps', 'rootkit', 'sqlattack', 'xterm'], 'U2R', inplace=True)
    return df

In [13]:
change_label(data_Train)

,duration,protocol_type,service,flag,src_bytes,dst_bytes,land,wrong_fragment,urgent,hot,...,dst_host_srv_count,dst_host_same_srv_rate,dst_host_diff_srv_rate,dst_host_same_src_port_rate,dst_host_srv_diff_host_rate,dst_host_serror_rate,dst_host_srv_serror_rate,dst_host_rerror_rate,dst_host_srv_rerror_rate,attack
0,0,udp,other,SF,146,0,0,0,0,0,...,1,0.00,0.60,0.88,0.00,0.00,0.00,0.00,0.00,normal
1,0,tcp,private,S0,0,0,0,0,0,0,...,26,0.10,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos
2,0,tcp,http,SF,232,8153,0,0,0,0,...,255,1.00,0.00,0.03,0.04,0.03,0.01,0.00,0.01,normal
3,0,tcp,http,SF,199,420,0,0,0,0,...,255,1.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,normal
4,0,tcp,private,REJ,0,0,0,0,0,0,...,19,0.07,0.07,0.00,0.00,0.00,0.00,1.00,1.00,Dos
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125967,0,tcp,private,S0,0,0,0,0,0,0,...,25,0.10,0.06,0.00,0.00,1.00,1.00,0.00,0.00,Dos
125968,8,udp,private,SF,105,145,0,0,0,0,...,244,0.96,0.01,0.01,0.00,0.00,0.00,0.00,0.00,normal
125969,0,tcp,smtp,SF,2231,384,0,0,0,0,...,30,0.12,0.06,0.00,0.00,0.72,0.00,0.01,0.00,normal
125970,0,tcp,klogin,S0,0,0,0,0,0,0,...,8,0.03,0.05,0.00,0.00,1.00,1.00,0.00,0.00,Dos


In [16]:
# # Assume data_train_with_features_selected and data_test_with_features_selected are your dataframes


# # Save data_test_with_features_selected to a CSV file
# data_Train.to_csv('dataWithNameAttack.csv', index=False)

# print("CSV files have been saved successfully.")

CSV files have been saved successfully.


In [27]:
data_KNN =pd.read_csv("data_test_top_10_features.csv")

In [28]:
# Boucler à travers chaque colonne pour afficher les valeurs uniques
for col in data_KNN.columns:
    unique_values = data_KNN[col].unique()
    print(f"Colonnes: {col}")
    print(f"Valeurs uniques: {unique_values}")
    print("\n")

Colonnes: same_srv_rate
Valeurs uniques: [0.01 1.   0.02 0.09 0.16 0.07 0.05 0.08 0.   0.5  0.06 0.03 0.25 0.12
 0.33 0.04 0.14 0.11 0.99 0.1  0.8  0.15 0.67 0.13 0.95 0.34 0.19 0.18
 0.17 0.2  0.24 0.27 0.38 0.75 0.86 0.29 0.6  0.92 0.37 0.23 0.94 0.78
 0.43 0.39 0.26 0.91 0.42 0.3  0.54 0.47 0.35 0.21 0.41 0.32 0.28 0.22
 0.56 0.62 0.36 0.85 0.31 0.98 0.4  0.46 0.55 0.52 0.83 0.48 0.65 0.9
 0.57 0.89 0.53 0.93 0.58]


Colonnes: flag_SF
Valeurs uniques: [0 1]


Colonnes: flag_S0
Valeurs uniques: [0 1]


Colonnes: dst_host_srv_serror_rate
Valeurs uniques: [0.   0.01 0.95 0.77 1.   0.39 0.02 0.37 0.75 0.66 0.38 0.65 0.8  0.05
 0.14 0.06 0.58 0.22 0.78 0.44 0.36 0.51 0.97 0.23 0.43 0.55 0.76 0.1
 0.72 0.82 0.67 0.34 0.03 0.56 0.28 0.79 0.29 0.98 0.84 0.4  0.47 0.41
 0.83 0.21 0.04 0.25 0.08 0.5  0.87 0.73 0.46 0.11 0.81 0.18 0.74 0.71
 0.16 0.93 0.2  0.45 0.52 0.49 0.07 0.48 0.62 0.69 0.64 0.42 0.3  0.59
 0.96 0.68 0.57 0.35 0.91 0.7  0.63 0.31 0.6  0.15 0.54 0.33 0.9  0.61
 0.27 0.92 0.